<a href="https://colab.research.google.com/github/2002yoons/AI_05_Yunse_1st_projoect/blob/main/AI_05_%EC%9D%B4%EC%9C%A4%EC%84%B8_S2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 분석


  

타겟은 지연도착 여부입니다. 그러므로 분류문제로 접근하겠습니다. 







##데이터 선정 이유

주어진 데이터는 항공정보 데이터입니다. 데이터를 이용해서 도착지연을 예측하는 것이 이번 프로젝트의 목표입니다.



1.   항공기 지연도착으로 인해 방송촬영 시간에 늦었던 경험

2. 코로나로 인해 늘어날 수요
  
3. 기후 변화로 인해 더욱 부각될 지연이슈






###문제정의

지연도착의 여부는 예,아니오로 구분됩니다, 분류문제로 접근하겠습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#데이터 분석에 필요한 데이터를 불러오겠습니다.
import pandas as pd

#2019년 2월 항공기 데이터입니다
df_2019 = pd.read_csv('/content/drive/MyDrive/프로젝트/Feb_2019_ontime.csv')

#2020년 2월 항공기 데이터입니다
df_2020 = pd.read_csv('/content/drive/MyDrive/프로젝트/Feb_2020_ontime.csv')

In [ ]:
# Month 피쳐를 생성해 2019년 데이터는 2019, 2020년 데이터는 2020으로 넣겠습니다.
df_2019['Year'] = 2019
df_2020['Year'] = 2020

#1월 2월 데이터를 합치겠습니다.
df = pd.concat([df_2019,df_2020],ignore_index=True)

In [ ]:
#데이터셋의 차원을 알아보겠습니다
df.shape 


In [ ]:
df.head()

In [ ]:
# 데이터 요약 통계량을 보겠습니다. 
df.describe(include=['number'],percentiles=[0.05,0.00,0.95,0.99]).T

In [ ]:
#문자열 피쳐의 요약 통계량을 보겠습니다
df.describe(include= ['object']).T


###데이터 스플릿

In [ ]:
# 데이터 누수를 방지하기 위해 먼저 테스트 (처음엔 먼저 데이터 프로세싱을 했는데, 하다보니 데이터 누수가 의심되어 처음부터 스플릿 하기. )
# 데이터가 110만개이기 떄문에 교차검증을 사용하지 않고 test,validate,train split으로 나누겠습니다.
from sklearn.model_selection import train_test_split
import numpy as np

train, test = train_test_split(df, test_size=0.2, random_state=2)
train, val = train_test_split(train,test_size=0.2,random_state=2)

print(df.shape)
print(train.shape)
print(test.shape)
print(val.shape)

###베이스라인 모델


In [ ]:
#타겟 확인 및 베이스 모델 선정, 가중치 계산
base = round(train['ARR_DEL15'].value_counts(normalize = True)[0],3)
ratio =  0.2/0.8
print(f'베이스 모델: {base}')
print(f'가중치: {ratio}')

##전처리















###결측치 확인

1.   결측치가 총 데이터에 차지하는 비율이 0.00004%로 제거조치해도 데이터셋에 영향을 주지 않을 양입니다, 다만 결측치가 포함된 특성이 ['TAIL_NUM'],['DEP_TIME'], ['DEP_DEL15'] , ['ARR_TIME'], ['ARR_DEL15']므로, 타겟과 밀접한 관계가 있습니다. 결측치가 포함된 데이터가 어떤 데이터인지 알아보았습니다. 

---





 2.   결측치를 포함한 모든 항공정보는 세가지 경우입니다:


   * 취소가 되어서 이륙시간, 착륙시간이 결측처리 된 경우 ----- 취소가 된 정보는 타겟과 무관하므로 전체 로우를 제거했습니다.
   * 우회가 되어서 지연착륙 여부를 알 수 없는 경우 ----- 우회가 된 정보는 상식적으로 지연착륙이 될 가능성이 크지만 타겟값이 결측되었습니다, 자문을 구할 수 없는 상황이므로 전체 로우를 제거했습니다.
   * 테일넘버가 결측되어,항공기를 식별할 수 없어 이륙시간, 착륙시간이, 지연착륙 여부가 결측된 경우 ----- 테일 넘버는 타겟과 무관하여 전체 로우를 제거했습니다. 

In [ ]:
#결측치의 비중을 확인하겠습니다
df_nan_ratio = pd.DataFrame(data=df.isnull().sum()/len(df),columns=['nan_ratio'])

a = pd.DataFrame(df_nan_ratio)
print(f'결측치를 가진 피쳐가 데이터셋에서 차지하는 비율 :{round(df_nan_ratio.values.mean()/100,5)}%')
a
#'TAIL_NUM','DEP_TIME','DEP_DEL15','ARR_TIME',ARR_DEL15' 총 5개의 피쳐에서 결측치를 발견했습니다, 평균 결측치 비중이 전체 데이터의 0.00004%이지만, 이륙시간과 착륙시간은 중요한 정보이므로, 결측치를 더 자세하게 알아보겠습니다. 

In [ ]:
#항공기의 정보가 담겼지만 착륙과 이륙에 결측치가 있다는 건 이륙이 취소되었음을 의심할 수 있습니다, 취소된 정보에 결측치는 없는지 확인하겠습니다.
def cleaning(df):
    #Unnamed:21 피쳐는 무의미하므로 제거하겠습니다.
    df.drop(columns = ['Unnamed: 21'],axis=1,inplace=True)

    pd.options.mode.chained_assignment = None  

    #취소된 항공정보는 '지연 착륙'타겟과 무관하므로 제거하겠습니다, 또한 우회한 항공정보는 상식적으로 지연 착륙 가능성이 크지만, 타겟이 전부 결측치므로, 우회 항공정보에 대한 지연착륙 여부를 파악할 수 없어 제거 조치하겠습니다. 
    df.drop(df[df['CANCELLED'] == 1.0].index,inplace = True)
    not_cancelled_but_with_nan = ((df['CANCELLED']==0.0) & (df.isnull().any(axis=1) == True))
    df.drop(df[not_cancelled_but_with_nan].index, inplace = True)

  
    # TAIL NUM 피쳐를 확인하겠습니다. TAIL NUM은 항공기의 고유 넘버입니다. 넘버 자체에 국가 또는 도시 정보를 볼 수 있지만, 총 5815개의 카디널리티가 존재해 일일히 분류하기 어렵습니다. 전체 특성을 삭제 조치하겠습니다.
    df.drop(columns =  ['TAIL_NUM'],inplace =True)
    df_nan_cleaning = pd.DataFrame(data=df.isnull().sum()/len(df),columns=['nan_ratio'])
    


    return df

In [ ]:
train = cleaning(train)
print(train.isnull().any().any())
val = cleaning(val)
print(val.isnull().any().any())
test = cleaning(test)
print(test.isnull().any().any())

##EDA

데이터의 인사이트를 알아보겠습니다.


1.   일주일 중 주말과 평일의 지연량이 눈에 띄게 다를까?
2.   어느 출발지에서 출발지연이 가장 많이 될까?
3.   어느 도착지에서 도착지연이 가장 많이 될까?
3.   지연착륙과 지연이륙이 어떤 관계일까?





In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# 일주일 중 어느 요일이 이용량이 가장 많을까?
plot_count= train['DAY_OF_WEEK'].value_counts()
fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(10)
fig.set_figheight(7)

plt.box(False)
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
sns.barplot(x=plot_count.index, y=plot_count.values,palette= 'crest')


In [ ]:
# 일주일 중 지연이 가장 많이 일어나는 날?
plot_del= train[train['ARR_DEL15']== 1.0]['DAY_OF_WEEK'].value_counts()
fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(10)
fig.set_figheight(7)

plt.box(False)
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
sns.barplot(x=plot_del.index, y=plot_del.values,palette= 'crest')

In [ ]:
#일주일 중 요일에 따른 지연율은 얼마나 될까?   (주말과 평일을 분리)
delay_ratio =plot_del/plot_count

fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(10)
fig.set_figheight(7)

plt.box(False)
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
sns.barplot(x=delay_ratio.index, y=delay_ratio.values,palette= 'crest')

In [ ]:
#타임 블록 중 지연이 제일 많이 발생하는 블록이 언제일까?
delay_block = train[['DEP_TIME_BLK','ARR_DEL15']].groupby('DEP_TIME_BLK').sum().sort_values(by='ARR_DEL15',ascending=False)
fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(18)
fig.set_figheight(7)

plt.box(False)
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
sns.barplot(x=delay_block.index, y=delay_block['ARR_DEL15'],palette= 'crest')

In [ ]:
#도착지에서 얼마나 많은 지연이 일어났는지?
delay_dest = train[['DEST','ARR_DEL15']].groupby('DEST').sum().sort_values(by='ARR_DEL15',ascending=False)[0:10]
fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(18)
fig.set_figheight(7)

plt.box(False)
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
sns.barplot(x=delay_dest.index, y=delay_dest['ARR_DEL15'],palette= 'crest')

In [ ]:
#출발지에서 얼마나 많은 지연출발이 일어났는지? 
delay_origin = train[['ORIGIN','DEP_DEL15']].groupby('ORIGIN').sum().sort_values(by='DEP_DEL15',ascending=False)[0:10]
fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(18)
fig.set_figheight(7)

plt.box(False)
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
sns.barplot(x=delay_origin.index, y=delay_origin['DEP_DEL15'],palette= 'crest')

### 특성 파악





####히트맵

In [ ]:
#
# multicollinearity를 확인하기 위해 특성 간 상관관계를 알아보겠습니다,

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (18, 16))
sns.heatmap(df.corr(), annot = True, cmap = 'PuBuGn')
plt.show()

#### 카이제곱검정

In [ ]:
from scipy.stats import chi2_contingency
obs_OP2 = pd.crosstab(train['OP_UNIQUE_CARRIER'], 
                            train['OP_CARRIER'],
                             margins = False)

c, p, dof, expected = chi2_contingency(obs_OP2) 

# Print the p-value

print(p)
obs_OP2

In [ ]:

obs_org = pd.crosstab(train['ORIGIN'], 
                            train['ORIGIN_AIRPORT_ID'],
                             margins = False)

c, p, dof, expected = chi2_contingency(obs_org) 

# Print the p-value
print(p)
obs_org

In [ ]:

obs_dest = pd.crosstab(train['DEST'], 
                            train['DEST_AIRPORT_ID'],
                             margins = False)

c, p, dof, expected = chi2_contingency(obs_dest) 

# Print the p-value
print(p)
obs_org

In [ ]:

obs_dest = pd.crosstab(train['OP_CARRIER_AIRLINE_ID'], 
                            train['OP_CARRIER_FL_NUM'],
                             margins = False)

c, p, dof, expected = chi2_contingency(obs_dest) 

# Print the p-value
print(p)
obs_org

### 특성 제거

In [ ]:
def deleting(df):

    # 다중공선성을 보이는 'ORIGIN_AIRPORT_SEQ_ID','DEST_AIRPORT_SEQ_ID'를 제거하겠습니다.(heatmap)
    df.drop(columns = ['ORIGIN_AIRPORT_SEQ_ID','DEST_AIRPORT_SEQ_ID'],inplace = True)

    #결측치 처리 과정에서 우회와 취소를 제거했으므로, 피쳐를 제거하겠습니다.(결측치 제거)
    df.drop(columns = ['DIVERTED','CANCELLED'],inplace = True)

    # ORIGIN_AIRPORT_ID 와 ORIGIN는 동일한 특성이므로 택일하겠습니다 (크로스탭)
    df.drop(columns = ['ORIGIN_AIRPORT_ID'],inplace = True)

    # DEST_AIRPORT_ID 와 DEST는 동일한 특성이므로 택일하겠습니다 (크로스탭)
    df.drop(columns = ['DEST_AIRPORT_ID'],inplace = True)

    #OP_CARRIER과 OP_UNIQUE_CARRIER, OP_CARRIER_AIRLINE_ID 셋 다 동일한 특성이므로 택일하겠습니다.(크로스탭)
    df.drop(columns = ['OP_UNIQUE_CARRIER','OP_CARRIER'],inplace = True)

    df.drop(columns = ['OP_CARRIER_AIRLINE_ID','OP_CARRIER_FL_NUM'],inplace = True)




    return df






In [ ]:
train = deleting(train)
val = deleting(val)
test = deleting(test)

###특성 생성

In [ ]:
#ARR_TIME_BLK 생성

def engineering(df):
    df.loc[(df['ARR_TIME']>=600) & (df['ARR_TIME']<=659),'ARR_TIME_BLK'] ='0600-0659'
    df.loc[(df['ARR_TIME']>=700) & (df['ARR_TIME']<=759),'ARR_TIME_BLK'] ='0700-0759'
    df.loc[(df['ARR_TIME']>=800) & (df['ARR_TIME']<=859),'ARR_TIME_BLK'] ='0800-0759'
    df.loc[(df['ARR_TIME']>=900) & (df['ARR_TIME']<=959),'ARR_TIME_BLK'] ='0900-0759'
    df.loc[(df['ARR_TIME']>=1000) & (df['ARR_TIME']<=1059),'ARR_TIME_BLK'] ='1000-1059'
    df.loc[(df['ARR_TIME']>=1100) & (df['ARR_TIME']<=1159),'ARR_TIME_BLK'] ='1100-1159'
    df.loc[(df['ARR_TIME']>=1200) & (df['ARR_TIME']<=1259),'ARR_TIME_BLK'] ='1200-1259'
    df.loc[(df['ARR_TIME']>=1300) & (df['ARR_TIME']<=1359),'ARR_TIME_BLK'] ='1300-1359'
    df.loc[(df['ARR_TIME']>=1400) & (df['ARR_TIME']<=1459),'ARR_TIME_BLK'] ='1400-1459'
    df.loc[(df['ARR_TIME']>=1500) & (df['ARR_TIME']<=1559),'ARR_TIME_BLK'] ='1500-1559'
    df.loc[(df['ARR_TIME']>=1600) & (df['ARR_TIME']<=1659),'ARR_TIME_BLK'] ='1600-1659'
    df.loc[(df['ARR_TIME']>=1700) & (df['ARR_TIME']<=1759),'ARR_TIME_BLK'] ='1700-1759'
    df.loc[(df['ARR_TIME']>=1800) & (df['ARR_TIME']<=1859),'ARR_TIME_BLK'] ='1800-1859'
    df.loc[(df['ARR_TIME']>=1900) & (df['ARR_TIME']<=1959),'ARR_TIME_BLK'] ='1900-1959'
    df.loc[(df['ARR_TIME']>=2000) & (df['ARR_TIME']<=2059),'ARR_TIME_BLK'] ='2000-2059'
    df.loc[(df['ARR_TIME']>=2100) & (df['ARR_TIME']<=2159),'ARR_TIME_BLK'] ='2100-2159'
    df.loc[(df['ARR_TIME']>=2200) & (df['ARR_TIME']<=2259),'ARR_TIME_BLK'] ='2200-2259'
    df.loc[(df['ARR_TIME']>=2300) & (df['ARR_TIME']<=2359),'ARR_TIME_BLK'] ='2300-2359'
    df.loc[(df['ARR_TIME']>=1) & (df['ARR_TIME']<=559),'ARR_TIME_BLK'] ='0001-0559'

    #일주일을 주말과 평일로 나눈 새로운 특성 생성
    df.loc[df['DAY_OF_WEEK']<=5,'WEEK_CAT'] = 'weekday'
    df.loc[df['DAY_OF_WEEK']>5,'WEEK_CAT'] = 'weekend'

    #시간 블록당 지연횟수 총합의 새로운 특성 생성 (merge이용)
    block_delay = df[['DEP_TIME_BLK','ARR_DEL15']].groupby('DEP_TIME_BLK').sum().sort_values(by='ARR_DEL15',ascending=False)
    df = df.merge(block_delay, left_on='DEP_TIME_BLK', right_on='DEP_TIME_BLK') 
    df.rename({'ARR_DEL15_y':'DEP_TIME_BLK_DELAY','ARR_DEL15_x':'ARR_DEL15' }, inplace=True, axis=1)

    # 출발지별 지연횟수 총합의 새로운 특성 생성
    delay_origin = df[['ORIGIN','DEP_DEL15']].groupby('ORIGIN').sum().sort_values(by='DEP_DEL15',ascending=False)[0:10]
    df = df.merge(delay_origin, left_on='ORIGIN', right_on='ORIGIN') 
    df.rename({'DEP_DEL15_y':'ORIGIN_DELAY','DEP_DEL15_x':'DEP_DEL15' }, inplace=True, axis=1)

    # 도착지별 지연횟수 총합의 새로운 특성 생성
    delay_dest = df[['DEST','ARR_DEL15']].groupby('DEST').sum().sort_values(by='ARR_DEL15',ascending=False)
    df = df.merge(delay_dest, left_on='DEST', right_on='DEST') 
    df.rename({'ARR_DEL15_y':'DEST_DELAY','ARR_DEL15_x':'ARR_DEL15' }, inplace=True, axis=1)

    #거리를 이산화시켜 네가지 분류 특성 생성
    df['DISTANCE_CAT'] = pd.qcut(df['DISTANCE'], q=4,labels=[1,2,3,4])

    #타입변환
    df['DEP_TIME'] = df['DEP_TIME'].astype(int)
    df['DEP_DEL15'] = df['DEP_DEL15'].astype(int)
    df['ARR_TIME'] = df['ARR_TIME'].astype(int)
    df['ARR_DEL15'] = df['ARR_DEL15'].astype(int)
    df['DISTANCE'] = df['DISTANCE'].astype(int)
    df['blck_delay'] = df['DEP_TIME_BLK_DELAY'].astype(int)
    df['delay_origin'] = df['ORIGIN_DELAY'].astype(int)
    df['delay_dest'] = df['DEST_DELAY'].astype(int)

    #카디널리티가 높은 특성은 제거하겠습니다.

    #df.drop(['DEP_TIME','ARR_TIME','DISTANCE','DEST'],axis = 1,inplace =True)

    



    return df

In [ ]:
train = engineering(train)
val = engineering(val)
test = engineering(test)

In [ ]:
train

#모델학습

1.   저희의 목표는 항공기의 지연을 예측하는 것입니다. 그러므로 타겟은 ['ARR_DEL15']입니다.




In [ ]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap



In [ ]:
from category_encoders import OrdinalEncoder
from category_encoders import TargetEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

###인코딩

In [ ]:
enc = OrdinalEncoder()
train_enc = enc.fit_transform(train)
val_enc = enc.transform(val)
test_enc = enc.transform(test)

###테스트셋 분리

In [ ]:
#타겟 선정, 학습 검증 테스트 데이터로 분할
target = 'ARR_DEL15'

X_train = train_enc.drop(columns=target)
y_train = train_enc[target]

X_val = val_enc.drop(columns=target)
y_val = val_enc[target]
X_test = test_enc.drop(columns=target)
y_test = test_enc[target]

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

###가중치 계산

In [ ]:
#가중치 계산
y_train.value_counts(normalize = True)
weight = len(y_train)/(2*np.bincount(y_train))
weight

In [ ]:
train.head()

###모델비교

In [ ]:
#렌덤포레스트와 XG부스트를 비교해보겠습니다.

pipe_RF = Pipeline([('preprocessing', make_pipeline(OrdinalEncoder())),('RF', RandomForestClassifier(n_estimators=100,max_depth=5, random_state=2))])
pipe_RF.fit(X_train,y_train)
y_val_pred_RF = pipe_RF.predict(X_val)
print(classification_report(y_val, y_val_pred_RF))

pipe_XGB = Pipeline([('preprocessing', make_pipeline(OrdinalEncoder())),('XGB', XGBClassifier(n_estimators=100,max_depth=5, random_state=2))])
pipe_XGB.fit(X_train,y_train)
y_val_pred_XGB = pipe_XGB.predict(X_val)
print(classification_report(y_val, y_val_pred_XGB))



###모델선택 및 특성중요도 계산

In [ ]:
# 특성 중요도를 위한 파이프라인 구축 계산 (permutation)
pipe = Pipeline([('preprocessing', make_pipeline(OrdinalEncoder())),('RF', RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1))])
pipe.named_steps

pipe.fit(X_train, y_train)

print('검증 정확도: ', pipe.score(X_val, y_val))

In [ ]:
#특성 중요도 계산(permutation importance)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    pipe.named_steps['RF'], 
    scoring='f1', 
    n_iter=30,  
    random_state=2
)

# permuter 계산은 preprocessing 된 X_val을 사용합니다.
X_val_transformed = pipe.named_steps['preprocessing'].transform(X_val)

# 실제로 fit 의미보다는 스코어를 다시 계산하는 작업입니다
permuter.fit(X_val_transformed, y_val)

feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

# 특성별 score 확인
eli5.show_weights(
    permuter, 
    top=None, 
    feature_names=feature_names # list 형식으로 넣음
)

In [ ]:
#permutaion 결과에 따라 음의 계수인 피쳐를 제거하겠습니다.
X_train.drop(['ARR_TIME_BLK','ARR_TIME'],axis=1,inplace=True)
X_val.drop(['ARR_TIME_BLK','ARR_TIME'],axis=1,inplace=True)
X_test.drop(['ARR_TIME_BLK','ARR_TIME'],axis=1,inplace=True)

###하이퍼 파라미터 튜닝

In [ ]:
#하이퍼 파라미터 튜닝을 하겠습니다
from sklearn.model_selection import RandomizedSearchCV

pipe = make_pipeline(
    OrdinalEncoder(),  
    RandomForestClassifier(random_state=2)
)


random_grid = {'randomforestclassifier__n_estimators': [200],
               'randomforestclassifier__max_features': ['auto'],
               'randomforestclassifier__max_depth': [5,10],
               'randomforestclassifier__min_samples_split': [2, 5],
               'randomforestclassifier__min_samples_leaf': [1, 2,4]
               }


rf_random = RandomizedSearchCV(estimator = pipe, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=1, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train);


In [ ]:
rf_random.best_params_

In [ ]:
#rf_random.best_params_
#{'randomforestclassifier__max_depth': 5,
 #'randomforestclassifier__max_features': 'auto',
 #'randomforestclassifier__min_samples_leaf': 4,
 #'randomforestclassifier__min_samples_split': 5,
 #'randomforestclassifier__n_estimators': 200}

###모델학습

In [ ]:
rf1 = RandomForestClassifier(class_weight= {False:weight[0],True:weight[1]}, max_depth = 10,max_features='auto',n_estimators = 200, min_samples_leaf = 4, min_samples_split= 5, random_state=2)


rf1.fit(X_train, y_train)
y_test_pred = rf1.predict(X_test)

print(classification_report(y_test, y_test_pred))

###모델 평가

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score

y_test_proba = rf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})

fig, ax = plt.subplots()
fig.set_facecolor('#f1f1f1')
fig.set_figwidth(12)
fig.set_figheight(12)

plt.box(False)
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='grey', linewidth=5)
plt.grid(True, color='grey')
plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');


auc_score = roc_auc_score(y_test, y_test_proba)
auc_score

## 모델해석

In [ ]:
model = rf.fit(X_train, y_train)


In [ ]:
import shap


In [ ]:
explainer = shap.TreeExplainer(model)




In [ ]:
choosen_instance = X_test.loc[[421]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:

choosen_instance = X_test.loc[[310]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)